# Image Effects

My attempt at recreating trivial photoshop effects + some custom visualizations I find cool.

## Imports

Let's get these out of the way.

In [ ]:
%load_ext autoreload
%autoreload 2

from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import os
from scipy.ndimage import gaussian_filter
from src.utils import load_rgb, save_gray, adjust_exponential, plot
import cv2
from sklearn.cluster import KMeans

## Grayscale

One might think grayscale is as easy as averaging the rgb channels but our eyes
perceive things differently. Thus a better way to grayscale is to use non-linear
weights when combining the rgb channels.

More here: https://en.wikipedia.org/wiki/Grayscale

In [ ]:
rgb = load_rgb("img/lenna.png")
gray = np.dot(rgb[...,:3], [0.299, 0.587, 0.114])
plot(1, 2, ("Original", rgb), ("Gray", gray))

## Hue shifting

Load image as hsv then shift all colors using a simple offset

In [ ]:
# Load colorful image.
rgb = load_rgb('img/food.jpg')
hsv = cv2.cvtColor(rgb, cv2.COLOR_RGB2HSV)

# Created shifted image.
hue = hsv[..., 0]
shift = 90
shifted_hue = (hue + shift) % 180
hsv[:, :, 0] = shifted_hue
shifted_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

# Plot comparison.
plot(1, 2, ("Original", rgb), ("Shifted Hue", shifted_rgb))

## Modeling macos dynamic backgrounds

Use gamma correction to emulate macos dynamic backgrounds.

In [ ]:
rgb = load_rgb("img/beach.jpg")
gammas = [1.3, 1, 0.7, 0.4]

divides = np.linspace(0, rgb.shape[1], 5).astype(int)
for start, stop, gamma in zip(divides, divides[1:], gammas):
    rgb[:,start:stop, :] = adjust_exponential(rgb[:,start:stop, :], gamma)

plot(1, 1, ("Modeling macos dynamic backgrounds", rgb))

## Shifting saturation

Description: Same concept as previous demo but messing with saturation instead of gamma

In [ ]:
rgb = load_rgb("img/beach.jpg")
hsv_exp = cv2.cvtColor(rgb, cv2.COLOR_RGB2HSV)
hsv_lin = cv2.cvtColor(rgb, cv2.COLOR_RGB2HSV)

saturations = [0.4, 0.7, 1, 1.3]
divides = np.linspace(0, hsv_exp.shape[1], len(saturations) + 1).astype(int)

for start, stop, saturation in zip(divides, divides[1:], saturations):
    hsv_lin[:,start:stop, 1] = hsv_lin[:,start:stop, 1] * saturation
    hsv_exp[:,start:stop, 1] = adjust_exponential(hsv_exp[:,start:stop, 1], saturation)

# Display.
plot(1, 3,
    ("rgb", rgb),
    ("Linear", cv2.cvtColor(hsv_lin, cv2.COLOR_HSV2RGB)),
    ("Exponential", cv2.cvtColor(hsv_exp, cv2.COLOR_HSV2RGB)),
    figsize=(10, 10)
)

## Color quantization + pixelation

Use color quantization + pixelation to convert new mario photo into old 1980s version

In [ ]:
# Load image.
rgb = load_rgb('img/mario.png')

# Create pixelated image.
orig_h, orig_w = rgb.shape[:2]
pixel_w, pixel_h = (25, 38)
down_sampled = cv2.resize(rgb, (pixel_w, pixel_h), interpolation=cv2.INTER_LINEAR)
pixelated = cv2.resize(down_sampled, (orig_w, orig_h), interpolation=cv2.INTER_NEAREST)

# Quantize the pixelated image.
n_colors = 7
arr = pixelated.reshape((-1, 3))
kmeans = KMeans(n_clusters=n_colors, n_init="auto").fit(arr)
labels = kmeans.labels_
centers = kmeans.cluster_centers_
quantized_pixelated = centers[labels].reshape(pixelated.shape).astype('uint8')

# Show original, pixelated, quantized_pixelated.
plot(1, 3,
    ("Original", rgb),
    ("Pixelated", pixelated),
    ("Quantized+Pixelation", quantized_pixelated),
)

---

In [ ]:
blur_garr = gaussian_filter(rgb, sigma=3)
plt.imshow(blur_garr)